<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/IonQ_%EC%96%91%EC%9E%90%ED%9A%8C%EB%A1%9C_6_%EC%96%91%EC%9E%90%EC%8B%9C%EB%AE%AC%EB%A0%88%EC%9D%B4%EC%85%98_and_Transpile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install qiskit-ionq
!pip install qiskit-aer

In [ ]:
#import IonQ backends
from qiskit_ionq import IonQProvider
ionq_provider = IonQProvider(token="xdNnUQmVTZyZ8ox7pHvPuU6qY5te9K8c")

#Qiskit imports
from qiskit import QuantumCircuit, Aer, execute, transpile
from qiskit.tools.visualization import array_to_latex, plot_histogram
from numpy import pi
import numpy as np

In [ ]:
hamiltonian = QuantumCircuit(4)
hamiltonian.h(0)
hamiltonian.rx(pi / 2, 1) #Implement R_x(pi / 2)

#CNOTs
for i in range(3):
  hamiltonian.cx(i, 3)
###for

hamiltonian.rz(pi / 3, 3) #R_z on ancilla

#CNOTs undo
for i in range(3):
  hamiltonian.cx( 2 - i, 3) #hamiltonian.cx(i, 3)도 가능
###for

#undo Hadamard and R_x rotation

hamiltonian.rx(-pi / 2, 1)
hamiltonian.h(0)

hamiltonian.draw()

In [ ]:
# Implement on eigenstate
qc = QuantumCircuit(4)
qc.h(0) #initializing eigenstate of X with eigenvalue 1
qc.rx(-pi / 2, 1) #initializing eigenstate of Y with eigenvalue 1

# Third qubit is already in eigenstate of Z with eigenvalue 1
qc = qc.compose(hamiltonian) # Add Hamiltonian to circuit

# Draw circuit
qc.draw()


In [ ]:
# Get Statevector
backend = Aer.get_backend('statevector_simulator') # Define backend
job = execute(qc, backend) # Get job object
result = job.result() # Get result
vector = result.get_statevector() # Get statevector

array_to_latex(vector / np.exp(-1j * (pi / 6))) # Display statevector

In [ ]:
# Plot histogram of counts
qc.measure_all()
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend)
result = job.result()

#counts = result.get_counts()
# Get counts of only first three qubits
counts = {key[::-1][:-1]:val for key, val in result.get_counts().items()}
plot_histogram(counts)


In [ ]:
backend = ionq_provider.get_backend('ionq_qpu') # choose your backend
job = execute(qc, backend) # get the job object

# Check if job is complete
from qiskit.providers.jobstatus import JobStatus
import time
while job.status() is not JobStatus.DONE:
  print("Job status is ", job.status())
  time.sleep(5)
###while

result = job.result() # get result object
counts = {key[::-1][:-1]:val for key, val in result.get_counts().items()}
plot_histogram(counts)

**Question 2: Qiskit Barrier Operation**


In [ ]:
# Part (i)
qc = QuantumCircuit(1)

qc.h(0)
qc.h(0)
qc.draw()

In [ ]:
# Transpile the circuit
transpiled_qc = transpile(qc, Aer.get_backend('qasm_simulator'))

# Draw the circuit
transpiled_qc.draw()



This is expected!! after all, H+ H = I

In [ ]:
# Part (ii) - Same circuit as in part (i), but with barrier in between Hadamard
new_qc = QuantumCircuit(1)
new_qc.h(0)
new_qc.barrier()
new_qc.h(0)
new_qc.draw()

In [ ]:
# Transpile the circuit
transpiled_qc = transpile(new_qc, Aer.get_backend('qasm_simulator'))

# Draw the circuit
transpiled_qc.draw()



중간에 barrier를 주었더니 결과가 위쪽과 달라요.

In [ ]:
# Part (iii)

circuit = QuantumCircuit(1)
circuit.rz(pi / 2, 0)
circuit.rx(pi / 4, 0)

circuit.draw()


In [ ]:
transpile(circuit, Aer.get_backend('qasm_simulator')).draw()

In [ ]:
# Part (iv)
circuit = QuantumCircuit(1)
circuit.rz(pi / 2, 0)
circuit.barrier()
circuit.rx(pi / 4, 0)

circuit.draw()
# Transpile and draw


In [ ]:
transpile(circuit, Aer.get_backend('qasm_simulator')).draw()